# iteration2

Data:
- interaktiot kaikkien muiden ryhmistä paitsi Alman kehittäjien
- interaktiot ryhmä, ei käyttäjäkohtaisia
- kaikki yritykset - konsernit mukana lisäämällä "K-" y-tunnuksen eteen
- metadatana perustietoa yrityksistä - numeeriset tilikausitiedot diskretisoitu tiettyihin persentiileihin (kts. muuttuja SELECTED_COMPANY_FEATURES alta)
- **data esikäsitelty proto2_data_prep-notebookissa**
- Warp-malli käytössä
- Minimiryhmäkoko = 2

Kysymyksiä:

1. Saadaanko näillä lisätyillä metatiedoilla mallista parempi metatietojen kanssa kuin ilman?
2. item_identity_features päälle vai pois?
3. Saadaanko lisätyillä metatiedoilla parempi precision?



## Importit

In [1]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
from lightfm.evaluation import reciprocal_rank
from lightfm.data import Dataset

import numpy as np
import pandas as pd

import statistics
import functools

WORKING_DIRECTORY = '/mnt/c/git/masters-thesis-code/jupyter/code/'

## Valitut metadatat yrityksille

In [2]:
SELECTED_COMPANY_FEATURES = ['company_form_code', 'location_municipality_code', 
                             'location_region_code', 'company_status_code', 'industry_code', 'turnover', 
                             'net_profit', 'personnel_average', 'performer_ranking_points', 'risk_rating_class']

## Ladataan yritysdata

In [3]:
COMPANIES_DF = pd.read_pickle(WORKING_DIRECTORY + "data/pandas_pickles/prod_data_proto2.pkl")

## Käsitellään yritysdataa LightFM:n Dataset-olion luontia varten

In [4]:
ITEM_IDS = list(COMPANIES_DF['business_id'].unique())

item_features_tmp = [COMPANIES_DF[feature].unique() for feature in SELECTED_COMPANY_FEATURES]

ITEM_FEATURE_LABELS = [item for sublist in item_features_tmp for item in sublist]

ITEM_FEATURES = [(company['business_id'], 
                  [company[feature] for feature in SELECTED_COMPANY_FEATURES])
                     for company in COMPANIES_DF.to_dict(orient='records')]

print(ITEM_FEATURES[0])
print(len(ITEM_FEATURES))
print(len(ITEM_FEATURE_LABELS))


('31431209', ['company_form_code+CO_26', 'location_municipality_code+091', 'location_region_code+01', 'company_status_code+AKT', 'industry_code+43', 'turnover+NaN', 'net_profit+NaN', 'personnel_average+NaN', 'performer_ranking_points+NaN', 'risk_rating_class+NaN'])
1337868
530


In [104]:
COMPANIES_DF[COMPANIES_DF['business_id'] == '01370820']

,business_id,company_name,company_form_code,location_municipality_code,location_region_code,company_status_code,industry_code,turnover,net_profit,personnel_average,performer_ranking_points,risk_rating_class
1333142,01370820,Leipomo Rosten Oy,company_form_code+CO_16,location_municipality_code+853,location_region_code+02,company_status_code+AKT,industry_code+10,turnover+0.98,net_profit+0.98,personnel_average+0.98,performer_ranking_points+0.6,risk_rating_class+GREEN


## Ladataan vuorovaikutusdata

In [7]:
interactions_tmp = pd \
    .read_csv(WORKING_DIRECTORY + 'data/interactions_2021_08_19.csv',
             delimiter='\t',
             dtype={
                 'group_id': 'string',
                 'business_id': 'string',
                 'owner': 'string'
             })
             
# otetaan pois 1 kokoiset ryhmät
group_sizes = interactions_tmp['group_id'].value_counts()
group_sizes_df = pd.DataFrame({'group_id': group_sizes.index, 'group_size': group_sizes.values})

INTERACTIONS_WITH_GROUP_SIZES_DF = interactions_tmp.merge(group_sizes_df, on='group_id')             
interactions_tmp = INTERACTIONS_WITH_GROUP_SIZES_DF[INTERACTIONS_WITH_GROUP_SIZES_DF.group_size >= 2]

interactions_tmp.sort_values('group_size')

,group_id,business_id,owner,group_size
155696,3e9dd356-2b21-45ae-9ee4-7cd6cc122fe1,07577937,5e87095492119e00066e7158,2
106198,31503959-943a-4081-abcc-dc80e5cb0402,15093748,5db034c64320cd0006d2b788,2
313746,cab22fae-db47-46b6-b902-3d9a1b1051f6,01163004,5e4534bc7bf061000697e940,2
313747,cab22fae-db47-46b6-b902-3d9a1b1051f6,10410900,5e4534bc7bf061000697e940,2
545392,0967d6ed-88b7-4023-a720-f09f7051f24d,17944788,5efdbc656488210007bc27f6,2
...,...,...,...,...
8042,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16029641,5e1489f3c2f568000654ecbb,3999
8043,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16030167,5e1489f3c2f568000654ecbb,3999
8044,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16030415,5e1489f3c2f568000654ecbb,3999
8031,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16001948,5e1489f3c2f568000654ecbb,3999


In [8]:
# lisätään konserniyrityksille interaktiot
concern_interactions = interactions_tmp.copy()
concern_interactions['business_id'] = 'K-' + concern_interactions['business_id'].astype(str)
concern_interactions = concern_interactions[concern_interactions.business_id.isin(ITEM_IDS)]
concern_interactions

,group_id,business_id,owner,group_size
5,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-01681709,60646431ae18cb00063ed63f,1862
6,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-15055514,60646431ae18cb00063ed63f,1862
7,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-01876143,60646431ae18cb00063ed63f,1862
9,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-05363070,60646431ae18cb00063ed63f,1862
10,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-01387534,60646431ae18cb00063ed63f,1862
...,...,...,...,...
548074,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-02106319,6110c56241e21e000857ca77,131
548110,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-20333371,6110c56241e21e000857ca77,131
548137,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-07027249,6110c56241e21e000857ca77,131
548162,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-02011774,6110c56241e21e000857ca77,131


In [9]:
# yhdistetään konserni-interaktiot tavallisiin ja poistetaan sellaiset interaktiot, joille ei löydy y-tunnusta
INTERACTIONS_DF = pd.concat([interactions_tmp, concern_interactions])
INTERACTIONS_DF = INTERACTIONS_DF[INTERACTIONS_DF.business_id.isin(ITEM_IDS)]
INTERACTIONS_DF[INTERACTIONS_DF['business_id'] == 'K-02011774']

,group_id,business_id,owner,group_size
18663,6c894b42-cbc6-4d18-8cf1-39ee91d2bf53,K-02011774,6058a712ae18cb00063ed639,2175
49204,66aea578-682d-45de-a200-77fa79a8c5e7,K-02011774,5db92c0ebc3e9100062ac0b0,3692
217786,608ac4ff-ab75-425b-a4f1-74d5defd43a3,K-02011774,5fbe6408f464de0006491d9e,1541
231830,4d2c2290-72be-40fe-8506-42718d2aac25,K-02011774,5f5613da9769490006b0ebb3,3685
254761,58f4cb31-0d40-42b2-9268-10d6dc64f1a0,K-02011774,5f5613da9769490006b0ebb3,2595
298418,2177a613-3b16-483e-9c9b-dc20bd4225f6,K-02011774,608863795602580007e70ddf,3925
354226,62c0017d-ff32-4584-8c85-562e9a1e8329,K-02011774,5fab9e2dc07a1900066bab26,3949
412123,3a44f374-56ee-4500-8829-c4b942b7afec,K-02011774,6033ae80484e8a0006fe437a,3966
439385,725e7cd0-7031-4988-b4cb-69672c611514,K-02011774,5db83a1cbc3e9100062ac0ab,1846
451491,acfdacd6-0de0-4568-a819-c3c2e15ef221,K-02011774,6033ae80484e8a0006fe437a,3809


In [10]:
def print_interactions_meta_data(interactions_df):
    print('ryhmiä: {groups}, interaktioita {interactions}, yrityksiä {companies}'
          .format(groups=len(list(interactions_df['group_id'].unique())),
                  interactions=interactions_df.shape[0], 
                  companies=len(list(interactions_df['business_id'].unique()))))


print('\n----- group_size>=2 -----')
print_interactions_meta_data(INTERACTIONS_DF)


----- group_size>=2 -----
ryhmiä: 1312, interaktioita 598703, yrityksiä 143839


## Luodaan LightFM:n ymmärtämät Dataset-oliot eri testitapauksille

In [11]:
def create_dataset(interactions_df, item_identity_features):
    dataset = Dataset(user_identity_features=False, item_identity_features=item_identity_features)
    
    interactions = [(interaction['group_id'], interaction['business_id']) 
                for interaction in interactions_df.to_dict(orient='records')]
    
    user_ids = list(set(interactions_df['group_id'].values))

    dataset.fit(users=user_ids, items=ITEM_IDS, item_features=ITEM_FEATURE_LABELS)
    
    (interactions_ds, weights_ds) = dataset.build_interactions(interactions)
    
    item_features_ds = dataset.build_item_features(ITEM_FEATURES, normalize=False)

    # USER_MAP_DS = dataset.mapping()[0]
    # ITEM_MAP_DS = dataset.mapping()[2]
    ITEM_FEATURE_MAP_DS = dataset.mapping()[3]
    print(len(ITEM_FEATURE_MAP_DS))
    
    return (interactions_ds, item_features_ds)    

In [12]:
# item_identity_features off
(INTERACTIONS_2_DS, ITEM_FEATURES_2_DS) = create_dataset(INTERACTIONS_DF, False)
INTERACTIONS_2_DS

530


<1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
	with 598703 stored elements in COOrdinate format>

In [13]:
# item_identity_fetures on
(INTERACTIONS_2_IDENTITY_DS, ITEM_FEATURES_2_IDENTITY_DS) = create_dataset(INTERACTIONS_DF, True)
INTERACTIONS_2_IDENTITY_DS

1338398


<1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
	with 598703 stored elements in COOrdinate format>

## Luodaan cross-validationia varten ositetut datasetit

In [14]:
def create_partitioned_datasets(interactions_ds):
    (rest, fifth_1) = random_train_test_split(interactions_ds, test_percentage=0.2)
    (rest, fifth_2) = random_train_test_split(rest, test_percentage=0.25)
    (rest, fifth_3) = random_train_test_split(rest, test_percentage=0.3333333)
    (fifth_4, fifth_5) = random_train_test_split(rest, test_percentage=0.5)
    
    return [fifth_1, fifth_2, fifth_3, fifth_4, fifth_5]
    

In [15]:
INTERACTIONS_2_CV = create_partitioned_datasets(INTERACTIONS_2_DS)
INTERACTIONS_2_CV

[<1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
 	with 119741 stored elements in COOrdinate format>,
 <1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
 	with 119741 stored elements in COOrdinate format>,
 <1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
 	with 119741 stored elements in COOrdinate format>,
 <1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
 	with 119740 stored elements in COOrdinate format>,
 <1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
 	with 119740 stored elements in COOrdinate format>]

In [16]:
INTERACTIONS_2_IDENTITY_CV = create_partitioned_datasets(INTERACTIONS_2_IDENTITY_DS)
INTERACTIONS_2_IDENTITY_CV

[<1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
 	with 119741 stored elements in COOrdinate format>,
 <1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
 	with 119741 stored elements in COOrdinate format>,
 <1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
 	with 119741 stored elements in COOrdinate format>,
 <1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
 	with 119740 stored elements in COOrdinate format>,
 <1312x1337868 sparse matrix of type '<class 'numpy.int32'>'
 	with 119740 stored elements in COOrdinate format>]

## Arvioidaan mallien laatua

In [17]:
NUM_THREADS = 8

def run_evaluation_function(model, test_ds, train_ds, evaluation_function, name, item_features=None):    
    print('Calculating {name} for train dataset...'.format(name=name))
    train_results = evaluation_function(model, train_ds, item_features=item_features, num_threads=NUM_THREADS)
    np.savetxt('iteration2-train-results-{}.txt'.format(name), train_results)
    train_metric = train_results.mean()
    
    print('Calculating {name} for test dataset...'.format(name=name))
    test_results = evaluation_function(model, test_ds, train_ds, item_features=item_features, num_threads=NUM_THREADS)
    np.savetxt('iteration2-test-results-{}.txt'.format(name), test_results)
    test_metric = test_results.mean()
    
    print('{name}: train {train_metric:.2f}, test {test_metric:.2f}'.format(name=name, 
                                                                            train_metric=train_metric, 
                                                                            test_metric=test_metric))
    print('\n')
    return (train_metric, test_metric)

def run_evaluations_for_ds(model, train_ds, test_ds, model_name, item_features=None):
    auc = run_evaluation_function(model, test_ds, train_ds, auc_score, 'AUC_' + model_name, item_features)
    precision = run_evaluation_function(model, test_ds, train_ds, precision_at_k, 'PRECISION_' + model_name, item_features)
    recall = run_evaluation_function(model, test_ds, train_ds, recall_at_k, 'RECALL_' + model_name, item_features)
    reciprocal = run_evaluation_function(model, test_ds, train_ds, reciprocal_rank, 'RECIPROCAL_' + model_name, item_features)
    
    return (auc, precision, recall, reciprocal)
    

def run_evaluations(name, interactions_cv, item_features_ds):
    
    results = {
        name: []
    }
    
    for i in range(0, len(interactions_cv)):
        print('Starting iteration {}...'.format(i))    
            
        test_ds = interactions_cv[i]
        
        # laitetaan uuteen listaan kaikki paitsi testidatasetti
        train_ds_tmp = [ds for j,ds in enumerate(interactions_cv) if j != i]
        # yhdistetään treenidatasetiksi valikoituneet vuorovaikutusmatriisit
        train_ds = functools.reduce(lambda a,b: a + b, train_ds_tmp)
        
        MODEL = LightFM(loss='warp')
        MODEL.fit(train_ds, item_features=item_features_ds, epochs=5, num_threads=NUM_THREADS, verbose=True)
        
        results[name].append(run_evaluations_for_ds(MODEL, train_ds, test_ds, '{}_{}'.format(name, i), item_features_ds))
        
    return results


In [18]:
def print_metric_result(result_arr, model_name):
    train_results = [x[0] for x in result_arr]
    test_results = [x[1] for x in result_arr]
    
    print('{name}:\n train mean {train_mean:.2f} ({train_arr})\n test mean {test_mean:.2f} ({test_arr})\n'
          .format(train_mean=statistics.mean(train_results),
                 test_mean=statistics.mean(test_results),
                 train_arr=['%.2f' % x for x in train_results],
                 test_arr=['%.2f' % x for x in test_results],
                 name=model_name))
    

def print_all_results(results):
    for i,metric in enumerate(['AUC', 'PRECISION', 'RECALL', 'RECIPROCAL']):
        print('\n-----{}-----'.format(metric))
        for model_name,result_arr in results.items():
            print_metric_result([res[i] for res in result_arr], model_name)
    

### Tulokset

In [19]:
RESULTS_IDENTITY_OFF = run_evaluations('IDENTITY_FEATURES_OFF', INTERACTIONS_2_CV, ITEM_FEATURES_2_DS)
RESULTS_IDENTITY_ON = run_evaluations('IDENTITY_FEATURES_ON', INTERACTIONS_2_IDENTITY_CV, ITEM_FEATURES_2_IDENTITY_DS)

Starting iteration 0...


Epoch: 100%|██████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.92it/s]


Calculating AUC_IDENTITY_FEATURES_OFF_0 for train dataset...
Calculating AUC_IDENTITY_FEATURES_OFF_0 for test dataset...
AUC_IDENTITY_FEATURES_OFF_0: train 0.99, test 0.99


Calculating PRECISION_IDENTITY_FEATURES_OFF_0 for train dataset...
Calculating PRECISION_IDENTITY_FEATURES_OFF_0 for test dataset...
PRECISION_IDENTITY_FEATURES_OFF_0: train 0.14, test 0.08


Calculating RECALL_IDENTITY_FEATURES_OFF_0 for train dataset...
Calculating RECALL_IDENTITY_FEATURES_OFF_0 for test dataset...
RECALL_IDENTITY_FEATURES_OFF_0: train 0.01, test 0.01


Calculating RECIPROCAL_IDENTITY_FEATURES_OFF_0 for train dataset...
Calculating RECIPROCAL_IDENTITY_FEATURES_OFF_0 for test dataset...
RECIPROCAL_IDENTITY_FEATURES_OFF_0: train 0.21, test 0.14


Starting iteration 1...


Epoch: 100%|██████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.38it/s]


Calculating AUC_IDENTITY_FEATURES_OFF_1 for train dataset...
Calculating AUC_IDENTITY_FEATURES_OFF_1 for test dataset...
AUC_IDENTITY_FEATURES_OFF_1: train 0.99, test 0.99


Calculating PRECISION_IDENTITY_FEATURES_OFF_1 for train dataset...
Calculating PRECISION_IDENTITY_FEATURES_OFF_1 for test dataset...
PRECISION_IDENTITY_FEATURES_OFF_1: train 0.14, test 0.09


Calculating RECALL_IDENTITY_FEATURES_OFF_1 for train dataset...
Calculating RECALL_IDENTITY_FEATURES_OFF_1 for test dataset...
RECALL_IDENTITY_FEATURES_OFF_1: train 0.01, test 0.01


Calculating RECIPROCAL_IDENTITY_FEATURES_OFF_1 for train dataset...
Calculating RECIPROCAL_IDENTITY_FEATURES_OFF_1 for test dataset...
RECIPROCAL_IDENTITY_FEATURES_OFF_1: train 0.22, test 0.15


Starting iteration 2...


...

Calculating AUC_IDENTITY_FEATURES_ON_2 for train dataset...
Calculating AUC_IDENTITY_FEATURES_ON_2 for test dataset...
AUC_IDENTITY_FEATURES_ON_2: train 1.00, test 0.99


Calculating PRECISION_IDENTITY_FEATURES_ON_2 for train dataset...
Calculating PRECISION_IDENTITY_FEATURES_ON_2 for test dataset...
PRECISION_IDENTITY_FEATURES_ON_2: train 0.18, test 0.10


Calculating RECALL_IDENTITY_FEATURES_ON_2 for train dataset...
Calculating RECALL_IDENTITY_FEATURES_ON_2 for test dataset...
RECALL_IDENTITY_FEATURES_ON_2: train 0.01, test 0.01


Calculating RECIPROCAL_IDENTITY_FEATURES_ON_2 for train dataset...
Calculating RECIPROCAL_IDENTITY_FEATURES_ON_2 for test dataset...
RECIPROCAL_IDENTITY_FEATURES_ON_2: train 0.27, test 0.18


Starting iteration 3...


Epoch: 100%|██████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.85s/it]


Calculating AUC_IDENTITY_FEATURES_ON_3 for train dataset...
Calculating AUC_IDENTITY_FEATURES_ON_3 for test dataset...
AUC_IDENTITY_FEATURES_ON_3: train 1.00, test 0.99


Calculating PRECISION_IDENTITY_FEATURES_ON_3 for train dataset...
Calculating PRECISION_IDENTITY_FEATURES_ON_3 for test dataset...
PRECISION_IDENTITY_FEATURES_ON_3: train 0.17, test 0.08


Calculating RECALL_IDENTITY_FEATURES_ON_3 for train dataset...
Calculating RECALL_IDENTITY_FEATURES_ON_3 for test dataset...
RECALL_IDENTITY_FEATURES_ON_3: train 0.01, test 0.01


Calculating RECIPROCAL_IDENTITY_FEATURES_ON_3 for train dataset...
Calculating RECIPROCAL_IDENTITY_FEATURES_ON_3 for test dataset...
RECIPROCAL_IDENTITY_FEATURES_ON_3: train 0.27, test 0.17


Starting iteration 4...


Epoch: 100%|██████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.99s/it]


Calculating AUC_IDENTITY_FEATURES_ON_4 for train dataset...
Calculating AUC_IDENTITY_FEATURES_ON_4 for test dataset...
AUC_IDENTITY_FEATURES_ON_4: train 1.00, test 0.99


Calculating PRECISION_IDENTITY_FEATURES_ON_4 for train dataset...
Calculating PRECISION_IDENTITY_FEATURES_ON_4 for test dataset...
PRECISION_IDENTITY_FEATURES_ON_4: train 0.18, test 0.09


Calculating RECALL_IDENTITY_FEATURES_ON_4 for train dataset...
Calculating RECALL_IDENTITY_FEATURES_ON_4 for test dataset...
RECALL_IDENTITY_FEATURES_ON_4: train 0.01, test 0.02


Calculating RECIPROCAL_IDENTITY_FEATURES_ON_4 for train dataset...
Calculating RECIPROCAL_IDENTITY_FEATURES_ON_4 for test dataset...
RECIPROCAL_IDENTITY_FEATURES_ON_4: train 0.30, test 0.17




In [20]:
print_all_results(RESULTS_IDENTITY_OFF)


-----AUC-----
IDENTITY_FEATURES_OFF:
 train mean 0.99 (['0.99', '0.99', '0.99', '0.99', '0.99'])
 test mean 0.99 (['0.99', '0.99', '0.99', '0.99', '0.99'])


-----PRECISION-----
IDENTITY_FEATURES_OFF:
 train mean 0.14 (['0.14', '0.14', '0.14', '0.15', '0.14'])
 test mean 0.08 (['0.08', '0.09', '0.08', '0.08', '0.08'])


-----RECALL-----
IDENTITY_FEATURES_OFF:
 train mean 0.01 (['0.01', '0.01', '0.01', '0.01', '0.01'])
 test mean 0.01 (['0.01', '0.01', '0.01', '0.01', '0.01'])


-----RECIPROCAL-----
IDENTITY_FEATURES_OFF:
 train mean 0.22 (['0.21', '0.22', '0.22', '0.23', '0.22'])
 test mean 0.15 (['0.14', '0.15', '0.15', '0.15', '0.15'])



In [21]:
print_all_results(RESULTS_IDENTITY_ON)


-----AUC-----
IDENTITY_FEATURES_ON:
 train mean 1.00 (['1.00', '1.00', '1.00', '1.00', '1.00'])
 test mean 0.99 (['0.99', '0.99', '0.99', '0.99', '0.99'])


-----PRECISION-----
IDENTITY_FEATURES_ON:
 train mean 0.18 (['0.18', '0.18', '0.18', '0.17', '0.18'])
 test mean 0.09 (['0.09', '0.09', '0.10', '0.08', '0.09'])


-----RECALL-----
IDENTITY_FEATURES_ON:
 train mean 0.01 (['0.01', '0.01', '0.01', '0.01', '0.01'])
 test mean 0.01 (['0.01', '0.01', '0.01', '0.01', '0.02'])


-----RECIPROCAL-----
IDENTITY_FEATURES_ON:
 train mean 0.28 (['0.27', '0.28', '0.27', '0.27', '0.30'])
 test mean 0.17 (['0.17', '0.18', '0.18', '0.17', '0.17'])

